<a href="https://colab.research.google.com/github/pcashman21/feral-cat-census/blob/main/src/notebooks/train_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script trains a YOLOv8 model to detect cats.  

The exact detection depends on the training set used.  One model is trained on a set of about 3920 images of cats, 980 of which are original (downloaded from Google's [OpenImages v7 database](https://storage.googleapis.com/openimages/web/index.html), and the rest are augmentations of the originals on a bounding-box level.  This dataset was generated by the [feral-cat-census project](https://universe.roboflow.com/paul-cashman-mxgwb/feral-cat-census/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true) in Paul Cashman's Roboflow account.

Another detection is of cat faces or heads only.  This is trained on a set of about 3600 images of cat faces and heads, almost all of them where you can see both eyes of the cat.  Of these, there are about 930 original images and the rest are transformations at the bounding box level.  This was generated from the Roboflow feral-cat-faces project.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!install zipfile
!unzip /content/feral-cat-faces.v1i.yolov8.zip

install: missing destination file operand after 'zipfile'
Try 'install --help' for more information.
Archive:  /content/feral-cat-faces.v1i.yolov8.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/009b10ebcb3b20c8_jpg.rf.a56a5737157d0e9fe1185c8e1e760fe2.jpg  
 extracting: test/images/00a185001cd97c48_jpg.rf.a67e763dab388669765183baebf38355.jpg  
 extracting: test/images/00cc2b5e5151feb8_jpg.rf.c9346e7813ff65d17274d83aa1d93094.jpg  
 extracting: test/images/00eb193f9be4c4a7_jpg.rf.4ea8237350932ab0964b475d29b6bd6e.jpg  
 extracting: test/images/00fe4d88ac3dd641_jpg.rf.14d1f7190df9c6facf76cee68d429d44.jpg  
 extracting: test/images/01ae76cfa43ffb36_jpg.rf.dbab54ee81e7f30f9f3afe692a981df1.jpg  
 extracting: test/images/03acfd1833e7328e_jpg.rf.2a21c1b625fe4352cea349bc16f83af1.jpg  
 extracting: test/images/03b274afbf8eb7d7_jpg.rf.5e830183360e1e799196f6305

In [3]:
!pip install ultralytics

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 9.9 MB/s eta 0:00:00


100%|██████████| 6.23M/6.23M [00:00<00:00, 75.6MB/s]


In [4]:
model_configuration_path = '/content/data.yaml'
epochs = 100
image_size = 640
number_of_epochs_with_no_improvement = 10
epochs_per_checkpoint = 10
seed = 42

In [5]:
# Train the model
results = model.train(data=model_configuration_path, epochs=epochs, imgsz=image_size, patience=number_of_epochs_with_no_improvement, save_period = epochs_per_checkpoint, seed = seed, verbose = True)

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=100, patience=10, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fa

In [6]:
results

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78d1e34df520>
fitness: 0.6289728578107866
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.59901])
names: {0: 'cat-face'}
plot: True
results_dict: {'metrics/precision(B)': 0.8985710960572891, 'metrics/recall(B)': 0.8262466759876087, 'metrics/mAP50(B)': 0.8986365465580611, 'metrics/mAP50-95(B)': 0.5990102257277561, 'fitness': 0.6289728578107866}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.6085980323053175, 'inference': 3.1417710806733816, 'loss': 0.0009600834179950018, 'postprocess': 2.1115938822428384}

In [7]:
model.export(format='saved_model',imgsz=640,keras=True)

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.30GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx', 'onnx2tf>=1.15.4,<=1.17.5', 'sng4onnx>=1.0.1', 'onnxsim>=0.4.33', 'onnx_graphsurgeon>=0.3.26', 'tflite_support', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 262.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.4/400.4 kB 305.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 340.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 178.1 MB/s eta 0:00:00
   

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 4.2s, saved as 'runs/detect/train/weights/best.onnx' (11.6 MB)
TensorFlow SavedModel: running 'onnx2tf -i "runs/detect/train/weights/best.onnx" -o "runs/detect/train/weights/best_saved_model" -nuo --non_verbose'
TensorFlow SavedModel: export success ✅ 44.2s, saved as 'runs/detect/train/weights/best_saved_model' (29.2 MB)

Export complete (46.2s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best_saved_model imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best_saved_model imgsz=640 data=/content/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best_saved_model'

In [8]:
# See https://github.com/googlecolab/colabtools/issues/3409
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
!cp -R '/content/runs/detect/train' '/content/gdrive/My Drive/Cat images/feral-cat-faces.v1i.yolov8/runs/detect'